#### Imports

In [1]:
#We are obtaining the bayesian networks from https://www.bnlearn.com/bnrepository

from pgmpy.readwrite import BIFReader
from pgmpy.inference import VariableElimination
import random
from asvFormula.bayesianNetworks.bayesianNetwork import *
from asvFormula.bayesianNetworks import networkSamplesPath
from asvFormula.datasetManipulation import *

In [22]:
import importlib

# Assuming 'asvFormula.bayesianNetworks.bayesianNetwork' was imported before
from asvFormula.bayesianNetworks.bayesianNetwork import *

# Reload the module
importlib.reload(asvFormula.bayesianNetworks.bayesianNetwork)

<module 'asvFormula.bayesianNetworks.bayesianNetwork' from '/home/echu/tesis/pasantia-BICC/asvFormula/bayesianNetworks/bayesianNetwork.py'>

##### Gum code

In [3]:
"""
Gum Code that might be needed later
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

def datasetFromBayesianNetwork(model, n):
    g = gum.BNDatabaseGenerator(model)
    g.setRandomVarOrder()
    g.drawSamples(n)
    return g.to_pandas()
    
How to get the posterior of a bayesian network
pred = gum.getPosterior(cancerBN,evs={"Pollution":'high',"Smoker":'True'},target='Cancer')
instance = gum.Instantiation(pred)
instance.setFirst()

for _ in range(pred.domainSize()):
    print(instance.todict(True))
    print(pred.get(instance))
    instance.inc()
 
    """

'\nGum Code that might be needed later\nimport pyAgrum as gum\nimport pyAgrum.lib.notebook as gnb\n\ndef datasetFromBayesianNetwork(model, n):\n    g = gum.BNDatabaseGenerator(model)\n    g.setRandomVarOrder()\n    g.drawSamples(n)\n    return g.to_pandas()\n    \nHow to get the posterior of a bayesian network\npred = gum.getPosterior(cancerBN,evs={"Pollution":\'high\',"Smoker":\'True\'},target=\'Cancer\')\ninstance = gum.Instantiation(pred)\ninstance.setFirst()\n\nfor _ in range(pred.domainSize()):\n    print(instance.todict(True))\n    print(pred.get(instance))\n    instance.inc()\n \n    '

### Create the network and the dataset

In [4]:
random.seed(10)

cancerNetworkPath = networkSamplesPath + "/cancer.bif"
#cancerNetworkPath = networkSamplesPath + "/cancer_tree.bif"

BNmodel = BIFReader(cancerNetworkPath).get_model()
BNInference = VariableElimination(BNmodel)

In [5]:
variableToPredict = "Pollution"

# Create a BNDatabaseGenerator object from the model
dataFromBN = datasetFromBayesianNetwork(BNmodel, 2000)
valuesPerFeature, encodedDataset = encodeCategoricalColumns(dataFromBN)
dtTreeClassifier = decisionTreeFromDataset(encodedDataset, variableToPredict , 2)

dtAsNetwork = obtainNetworkXTreeStructure(dtTreeClassifier, encodedDataset.columns)

rootNodes = [node for node in dtAsNetwork.nodes if isRoot(node, dtAsNetwork)] 
root = rootNodes[0]

  0%|          | 0/5 [00:00<?, ?it/s]

The model accuracy is : 0.8975


### Compute the mean prediction of the decision tree

In [25]:
meanForDTinBN(dtTreeClassifier, BNInference, valuesPerFeature, variableToPredict, 'low')

0.9

In [ ]:
BNInference.query(variables=['Cancer']).values

In [5]:
query = BNInference.query(evidence={'Pollution':'high', 'Smoker':'True', 'Cancer' : 'True', 'Dyspnoea' : 'True'}, joint=True, variables = ['Xray'] )
feature = 'Xray'

In [ ]:
query.get_value(**{feature :'negative'})